In [10]:
import numpy as np
from scipy.linalg import expm

# Number of qubits
n_qubits = 4
i = 9 # encoding for psi_0
j = 2 # bitflip index for psi_f_noisy
dim = 2 ** n_qubits  # Hilbert space dimension
noise = 0.2

# Define identity and Pauli matrices
I = np.eye(2, dtype=np.complex64)
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex64)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)

# Build multi-qubit Hamiltonians (Pauli-Z and Pauli-X sums)
H0 = sum(np.kron(np.kron(np.eye(2**i), Z), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))
Hf = sum(np.kron(np.kron(np.eye(2**i), X), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))

# Time evolution function
def time_evolution(H, dt):
    return expm(-1j * H * dt)

# Adiabatic evolution function
def hamiltonian_evolution(psi, H_init, H_final, backwards=False, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * H_init + s * H_final
        U = time_evolution(H, dt if not backwards else -dt)
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)  # Normalize

# Start with a known initial qubit state |ψ₀⟩
true_psi_0 = np.zeros(dim, dtype=np.complex64)
true_psi_0[i] = 1

print(true_psi_0)

# Evolve forward to get final state |ψ_f⟩
psi_f = hamiltonian_evolution(true_psi_0, H0, Hf)

def apply_kraus_to_density_matrix(rho, kraus_ops):
    return sum(K @ rho @ K.conj().T for K in kraus_ops)

# Define noise (Depolarizing Channel)
def depolarizing_channel(p):
    K0 = np.sqrt(1 - (3 * p / 4)) * I
    K1 = np.sqrt(p / 4) * X
    K2 = np.sqrt(p / 4) * Y
    K3 = np.sqrt(p / 4) * Z

    return [K0, K1, K2, K3]

def apply_kraus_to_multi_qubit_density_matrix(rho, kraus_ops, qubit_index, n_qubits):
    dim = 2 ** n_qubits  # Full Hilbert space dimension
    rho_new = np.zeros((dim, dim), dtype=complex)

    for K in kraus_ops:
        # Expand Kraus operator to the full Hilbert space
        I_before = np.eye(2 ** qubit_index, dtype=complex)
        I_after = np.eye(2 ** (n_qubits - qubit_index - 1), dtype=complex)
        K_full = np.kron(np.kron(I_before, K), I_after)  # Expand to full space

        # Apply the expanded Kraus operator to the density matrix
        rho_new += K_full @ rho @ K_full.conj().T

    return rho_new

# Define the depolarizing channel for a single qubit
kraus_ops = depolarizing_channel(noise)  # Depolarizing channel

# Apply noise to a specific qubit (e.g., qubit 2 in a 5-qubit system)
rho_0 = np.outer(psi_f, psi_f.conj())
rho_noisy = apply_kraus_to_multi_qubit_density_matrix(rho_0, kraus_ops, j, n_qubits)

Hn = Hf + rho_noisy
# Evolve forward to get final state |ψ_f⟩
psi_f_noisy = hamiltonian_evolution(true_psi_0, H0, Hn)
#psi_f_noisy = psi_f

noise_fidelity = np.abs(np.vdot(psi_f_noisy, psi_f))**2

# Step 4: Define Grover search components

# Define computational basis states
basis_states = np.eye(dim, dtype=np.complex64)

# Define uniform superposition state |s⟩
psi_s = np.ones(dim, dtype=np.complex64) / np.sqrt(dim)

# Construct Diffusion Operator: D = 2|s⟩⟨s| - I
diffusion = 2 * np.outer(psi_s, psi_s.conj()) - np.eye(dim)

# Modify Oracle to Evolve Backwards Instead of Knowing Winning State
# Estimates |ψ₀⟩ by evolving |ψ_f_noisy⟩ backward
estimated_psi_0 = hamiltonian_evolution(psi_f_noisy, Hf, H0, backwards=True)

# Generate new oracle using backward evolution
oracle = np.eye(dim) - 2 * np.outer(estimated_psi_0, estimated_psi_0.conj())

# Number of Grover Iterations (~π/4 * sqrt(N))
num_iterations = int(np.floor(np.pi / 4 * np.sqrt(dim)))

# Initialize search state in |s⟩ (equal superposition of all states)
psi_search = psi_s.copy()

# Apply Grover iterations
for i in range(num_iterations):
    psi_search = oracle @ psi_search  # Apply Oracle
    psi_search = diffusion @ psi_search  # Apply Diffusion
    print(f"Iteration {i+1}: {psi_search}")

# Measure the final state by finding the largest amplitude
winning_index = np.argmax(np.abs(psi_search)**2)

# Reconstruct psi_0
winning_state = np.zeros(dim, dtype=np.complex64)
winning_state[winning_index] = 1

# Compute Fidelity with true |ψ₀⟩
fidelity = np.abs(np.dot(winning_state.conj().T, true_psi_0))**2

# Print results
print("\n# Step 5: Compare Results")
print("\nTrue initial state |ψ₀_true⟩:\n", true_psi_0)
print("\nEvolved final state |ψ_f⟩:\n", psi_f)
print("\nNoisy final state |ψ_f_noisy⟩:\n", psi_f_noisy)
print("Fidelity |ψ^noise_f⟩ with true |ψ_f⟩:", noise_fidelity)
print("\nEstimate |ψ₀⟩ :\n", estimated_psi_0)
print("\nReconstructed initial state |ψ₀_reconstructed⟩:\n", winning_state)
print("\nFidelity with true |ψ₀⟩:", fidelity)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Iteration 1: [0.20649631+0.00302836j 0.18574127-0.01280089j 0.21455099+0.00118602j
 0.21174736+0.00651104j 0.21482108+0.00137752j 0.2115264 +0.00746108j
 0.2127721 -0.00084903j 0.21473311-0.00059971j 0.18574127-0.01280089j
 0.59281442-0.04278886j 0.21174736+0.00651104j 0.19916388+0.01610584j
 0.2115264 +0.00746108j 0.19600418+0.01697312j 0.21473311-0.00059971j
 0.21698507+0.003824j  ]
Iteration 2: [0.13213218+0.00560413j 0.09372391-0.02368872j 0.14703779+0.00219479j
 0.14184952+0.01204901j 0.14753762+0.00254916j 0.14144063+0.01380712j
 0.14374586-0.00157118j 0.14737482-0.00110979j 0.09372391-0.02368872j
 0.84703401-0.07918301j 0.14184952+0.01204901j 0.11856315+0.0298047j
 0.14144063+0.01380712j 0.11271595+0.03140965j 0.14737482-0.00110979j
 0.15154219+0.00707651j]
Iteration 3: [ 0.03802119+0.00734238j -0.01230029-0.03103632j  0.05755011+0.00287555j
  0.05075259+0.01578629j